In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

## Data preprocessing

In [5]:
tfds.disable_progress_bar()

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    name='cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

### Format the Data

In [17]:
IMG_HEIGHT = 160
IMG_WIDTH = 160

def format_example(image, label):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (IMG_HEIGHT, IMG_WIDTH))
    return image, label

In [19]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [20]:
# Global setting
BATCH_SIZE = 32
BUFFER_SIZE = 1000

In [46]:
train_batches = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

## Create the base model from the pre-trained convnets

Create the base model from the `MobileNet V2`

In [28]:
IMG_SHAPE = [IMG_HEIGHT, IMG_WIDTH, 3]

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

This feature extractor convert each `160 x 160 x 3` image into a  
`5 x 5 x 1280` block of features.

In [27]:
base_model.predict(train_batches.take(1)).shape

(32, 5, 5, 1280)

## Create model

In [34]:
# Freeze the convolutional base
base_model.trainable = False

In [82]:
# Add Gloabl average layer to reduce 
# `BATCH_SIZE x 5 x 5 x CHANNEL` to `BATCH_SIZE x CHANNEL`
# Add Dense layer as classification head
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAvgPool2D(),
    tf.keras.layers.Dense(1)
])

In [83]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [86]:
(tf.data.experimental.cardinality(train_batches) == tf.data.experimental.UNKNOWN_CARDINALITY).numpy()
print('Means that TF does not know `train_batches` cardinality, so it cannot verify the total step of current dataset. Then it cannot know whether to generate a new iterator on the dataset ( Keras uses `(steps_per_epoch is None) or (cardinality == steps_per_epoch)` to dicide whether generate a new iter). So, Keras will not create a new iterator in this case.')

Means that TF does not know `train_batches` cardinality, so it cannot verify the total step of current dataset. Then it cannot know whether to generate a new iterator on the dataset ( Keras uses `(steps_per_epoch is None) or (cardinality == steps_per_epoch)` to dicide whether generate a new iter). So, Keras will not create a new iterator in this case.


In [84]:
# Train model
model.fit(
    train_batches,
    epochs=10,
    validation_data=(validation_batches),
#     steps_per_epoch=metadata.splits['train'].num_examples*0.8 // BATCH_SIZE
)

Epoch 1/10
582/582 [==============================] - 35s 60ms/step - loss: 0.2126 - accuracy: 0.9021 - val_loss: 0.0857 - val_accuracy: 0.9712
Epoch 2/10
582/582 [==============================] - 33s 56ms/step - loss: 0.0725 - accuracy: 0.9761 - val_loss: 0.0627 - val_accuracy: 0.9781
Epoch 3/10
582/582 [==============================] - 34s 58ms/step - loss: 0.0578 - accuracy: 0.9805 - val_loss: 0.0554 - val_accuracy: 0.9802
Epoch 4/10
582/582 [==============================] - 34s 59ms/step - loss: 0.0517 - accuracy: 0.9817 - val_loss: 0.0519 - val_accuracy: 0.9811
Epoch 5/10
582/582 [==============================] - 34s 59ms/step - loss: 0.0485 - accuracy: 0.9829 - val_loss: 0.0502 - val_accuracy: 0.9819
Epoch 6/10
582/582 [==============================] - 35s 61ms/step - loss: 0.0462 - accuracy: 0.9833 - val_loss: 0.0489 - val_accuracy: 0.9828
Epoch 7/10
582/582 [==============================] - 35s 59ms/step - loss: 0.0445 - accuracy: 0.9839 - val_loss: 0.0479 - val_accuracy:

## Fine tuning

In [87]:
# Un-freeze the top layers of the model
base_model.trainable = True
# Fine-tune from this layer onwards
# base_model has 155 total layers
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [90]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train_batches,
    validation_data=validation_batches,
    epochs=20,
    initial_epoch=10
)

Epoch 11/20
582/582 [==============================] - 39s 68ms/step - loss: 0.1209 - accuracy: 0.9550 - val_loss: 0.0707 - val_accuracy: 0.9802
Epoch 12/20
582/582 [==============================] - 38s 65ms/step - loss: 0.0672 - accuracy: 0.9738 - val_loss: 0.0517 - val_accuracy: 0.9824
Epoch 13/20
582/582 [==============================] - 37s 63ms/step - loss: 0.0511 - accuracy: 0.9805 - val_loss: 0.0481 - val_accuracy: 0.9841
Epoch 14/20
406/582 [===================>..........] - ETA: 10s - loss: 0.0404 - accuracy: 0.9855